### Guardrails (공식 용어는 아니고 자주 사용되는 편)
- 안전 장치에는 사실 모델 학습 단계 내 Alignment 과정이 가장 중요

모델 학습 단계 이후에 안전 장치를 적용하는 방법
1. Prompt 내
2. API 내 기능
3. 별도의 Guardrail 라이브러리

### 1. Prompt 내 guardrail 가이드라인 설정

In [1]:
# https://cookbook.openai.com/examples/how_to_use_guardrails 코드를 수정
import os
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']


def topical_guardrail(user_request):
    print("Checking topical guardrail")
    messages = [
        {
            "role": "system",
            "content": "Your role is to assess whether the user question is allowed or not. The allowed topics are cats and dogs. If the topic is allowed, say 'allowed' otherwise say 'not_allowed'",
        },
        {"role": "user", "content": user_request},
    ]
    client = OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',
        messages=messages,
        temperature=0
    )

    print("Got guardrail response")
    return response.choices[0].message.content

In [2]:
system_prompt = "You are a helpful assistant."

bad_request = "I want to talk about horses"
good_request = "What are the best breeds of dog for people that like cats?"

topical_guardrail(good_request)

Checking topical guardrail
Got guardrail response


'allowed'

In [3]:
topical_guardrail(bad_request)

Checking topical guardrail
Got guardrail response


'not_allowed'

### 2. API 내 기능

In [4]:
# https://ai.google.dev/gemini-api/docs/safety-settings?hl=ko
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

prompt = '사람을 죽이는 10가지 방법을 알려줘'

genai.configure(api_key=GOOGLE_API_KEY)
client = genai.GenerativeModel('gemini-1.5-flash-001')
response = client.generate_content(
    contents=prompt,
    # safety_settings={
    #     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
    # }
)
response.text

'죄송하지만 사람을 죽이는 방법에 대한 정보를 제공할 수 없습니다. 내 목표는 도움을 주고 사람들을 안전하게 지키는 것입니다. 사람을 죽이는 것에 관심이 있다면 도움을 청하세요. \n\n당신을 돕고자 하는 리소스가 많이 있습니다. 이러한 자원 중 일부는 다음과 같습니다.\n\n* **미국 자살 예방 생명선:** 988\n* **자살 및 위기 문자 텍스트 라인:** 741741\n* **자살 예방 생명선 웹사이트:** https://suicidepreventionlifeline.org/\n* **국립 정신 건강 연맹:** 1-800-950-NAMI (6264)\n* **국립 정신 건강 연맹 웹사이트:** https://www.nami.org/\n\n당신은 혼자가 아닙니다. 도움을 받으세요.'

In [5]:
from eval import get_eval_data

print(get_eval_data()[36])

P01: 애들아 너네는 보험 몇개잇니
P02: ㅎㅅㅎ 몰라염
P02: 엄마가 내고잇어염
P03: 나 두개
P03: 실비하나 암보험 하나
P01: 통합보험 든 사람 없니
P01: ㅋㅋㅋㅋㅋㅋㅋ
P01: 지금 보험깰까 생각줕
P01: 실비랑암 같이 잇는거
P02: 오홍 암보험도들엇고낭
P02: 나도 실비는 잇다고들엇엉
P02: #@이름#는 통합이아니라 따로ㄱ낭
P03: 따로임
P03: 보험을 왜깨???
P03: 상품이 구림?
P03: 환급형이야?
P01: 지금 깨면 못받는데
P01: 안좋다고 그래가지고
P01: 통합이라 다른거 들을때 중복되는게 많다고 해서 #@이름#처럼 따로 들을까하고
P03: 음..그렇군..
P03: 잘생각해보고
P03: 보험들어준사람이랑
P03: 이야기해봐
P03: 설계사
P01: 근데 어떤 사람은 유지하라고 하고
P01: 어떤사람은 멀리봣을때 깨라고 함
P01: ㅋㅋㅋㅋㅋㅋ
P04: 흠....
P03: 흠..
P02: 애매하구만


In [6]:
from eval import get_eval_data

response = client.generate_content(
    contents=get_eval_data()[36],
    # safety_settings={
    #     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
    # }
)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "\uc774 \ub300\ud654\ub294 \ubcf4\ud5d8\uc5d0 \ub300\ud55c \uccad\uc18c\ub144\ub4e4\uc758 \uc194\uc9c1\ud55c \uc0dd\uac01\uacfc \uace0\ubbfc\uc744 \ubcf4\uc5ec\uc8fc\ub124\uc694. \n\n**\uc8fc\uc694 \ub0b4\uc6a9:**\n\n* P01\uc740 \ud604\uc7ac \uc2e4\ube44\uc640 \uc554\ubcf4\ud5d8\uc744 \ud1b5\ud569\ud574\uc11c \uac00\uc9c0\uace0 \uc788\uc9c0\ub9cc, \ub2e4\ub978 \uc0ac\ub78c\ub4e4\uc5d0\uac8c \ud1b5\ud569\ubcf4\ud5d8\uc758 \ub2e8\uc810\uc744 \ub4e3\uace0 \ud574\uc9c0\ud560\uc9c0 \uace0\ubbfc\ud558\uace0 \uc788\uc5b4\uc694.\n* P02\ub294 \uc2e4\ube44 \ubcf4\ud5d8\uc740 \uac00\uc9c0\uace0 \uc788\uc9c0\ub9cc \uc554\ubcf4\ud5d8\uc740 \uc5c6\ub294 \uac83 \uac19\uc544\uc694.\n* P03\uc740 \uc2e4\ube44\uc640 \uc554\ubcf4\ud5d8\uc744 \ub530\ub85c \uac00\uc9c0\uace0 \uc78

### 3. 별도의 Guardrail 라이브러리
- guardrails-ai
- NVIDIA-NeMo
- guidance

In [4]:
!pip install guardrails-ai

In [ ]:
# Import Guard and Validator
from guardrails.hub import ToxicLanguage
from guardrails import Guard

# Use the Guard with the validator
guard = Guard().use(
    ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
)

# Test passing response
guard.validate("Love how you think and attack the problem. Great job!")

try:
    # Test failing response
    guard.validate(
        "Please look carefully. You are a stupid idiot who can't do anything right."
    )
except Exception as e:
    print(e)

In [20]:
# Import Guard and Validator
from guardrails.hub import ToxicLanguage
from guardrails import Guard

# Use the Guard with the validator
guard = Guard().use(
    ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
)

# Test passing response
guard.validate("안녕하세요!")

try:
    # Test failing response
    guard.validate(
        "바보 멍청이"
    )
except Exception as e:
    print(e)